In [1]:
# Importing pandas and seaborn libraries for data manipulation and charting
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

In [2]:
# Import data from CSV file
data = pd.read_excel('churn.xlsx')

In [3]:
# Function to clean column names

def column_name_remove_space(df):
    for x in df.columns:
        if " " in x:
            df = df.rename(columns={x:x.replace(" ","_").replace("(","")
                                    .replace(")","").replace(",","_").replace("/","_")})
    return df

Modelo Inicial

In [4]:
# Following columns have categorical data types and they need to be converted to dummy variables
dummy_variable_columns = ['GENDER','BILLING_PLATFORM','HAS_FREE_TRIAL','SUBSCRIPTION_PERIOD','BY_CAMPAIGN',
                          'SUBSCRIPTION_TYPE','HAS_ANY_KIDS_PROFILE_STREAMING','PRODUCT_SWITCH','IS_RECONNECT_PAID_TO_PAID','IS_RECONNECT_PAID_TO_TRIAL',
                          'IS_RECONNECT_TRIAL_TO_PAID','IS_RECONNECT_TRIAL_TO_TRIAL','STREAM_SPORTS','STREAM_SERIES',
                          'STREAM_MOVIES','PAYMENT_METHOD','PURCHASE_DEVICE_TYPE', 'CHURN']


# Following columns have continuous numerical data, which needs to be scaled before predictive model can be created.
# Hence, these columns will be transformed to normalized form.

to_transform = ['DISTINCT_TITLES','DAYS_SINCE_LAST_STREAM  ','MINS_WATCHED','DEVICE_COUNT',
                'DAYS_STREAM','NUMB_OF_PROFILES','CHILDREN','AGE_IN_MONTHS']

dummy_data_file_index = 0

In [5]:
from sklearn.preprocessing import StandardScaler

def get_dummy_data_with_output(dummy_variable_columns, data):
    global dummy_data_file_index
    dummy_data = pd.get_dummies(data, prefix=dummy_variable_columns, columns=dummy_variable_columns, drop_first= True)
    dummy_data = column_name_remove_space(dummy_data)
    dummy_data.to_csv('dummy_var_data'+str(dummy_data_file_index)+'.csv', index=False)
    y = dummy_data['CHURN_Yes']
    dummy_data.drop(columns=['CHURN_Yes'], inplace=True)
    dummy_data_file_index += 1
    return y, dummy_data

y, dummy_data = get_dummy_data_with_output(dummy_variable_columns, data)

In [6]:
# Split the training and test set 7:3 
X_train, X_test, y_train, y_test = train_test_split(dummy_data,y,test_size=.3, random_state=0)

In [7]:
# Creating a function to create prediction model based on statsmodel library and
# print details like model Summary, Confusion Matrix and Accuracy Score based on predicted values using test set
figsize=(8,5)

def get_sm_model(y_train,X_train, X_test):
    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)
    model=sm.OLS(y_train,X_train)
    model=model.fit()
    print(model.summary())
    y_hat = model.predict(X_test)
    arr = np.c_[y_hat, y_test]
    print('\nPrinting predicted and actual values:\n',arr)
    print('\nPrinting Mean Squared Error:\n',(np.sqrt(mean_squared_error(y_test, y_hat))))
    print('\nR2 Score:\n',r2_score(y_test, y_hat))

    return model, arr


In [8]:
# Creating a function to create prediction model based on sklearn library and
# print details like model Summary, Confusion Matrix and Accuracy Score based on predicted values using test set

def get_sk_model(y_train,X_train, X_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    arr = np.c_[X_train.columns.tolist(),model.coef_]
    intercept = model.intercept_
    print('\nPrinting model coefficients and intercept summary for sklearn model:\n',arr, model.intercept_)
    y_pred = model.predict(X_test)
    print('\nPrinting predicted and actual values from sklearn:\n',np.c_[y_pred, y_test])
    print('\nPrinting Mean Squared Error from sklearn:\n',(np.sqrt(mean_squared_error(y_test, y_pred))))
    print('\nR2 Score  from sklearn:\n',r2_score(y_test, y_pred))
    return model, arr, intercept

In [9]:
# Run statsmodel function
model_sm, arr_sm = get_sm_model(y_train,X_train, X_test);

                            OLS Regression Results                            
Dep. Variable:              CHURN_Yes   R-squared:                       0.494
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     212.7
Date:                Mon, 12 Sep 2022   Prob (F-statistic):               0.00
Time:                        14:16:06   Log-Likelihood:                -1816.7
No. Observations:                7000   AIC:                             3699.
Df Residuals:                    6967   BIC:                             3926.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [10]:
# Run sklearn model on reduced data set
model, arr, intercept = get_sk_model(y_train,X_train, X_test);


Printing model coefficients and intercept summary for sklearn model:
 [['CHILDREN' '-0.009210058977570388']
 ['AGE_IN_MONTHS' '0.008695015455023419']
 ['DEVICE_COUNT' '-0.0006620285818183069']
 ['DAYS_STREAM' '-0.0003886320445743644']
 ['NUMB_OF_PROFILES' '0.0037872567028301458']
 ['DISTINCT_TITLES' '-0.032033901565070536']
 ['DAYS_SINCE_LAST_STREAM' '0.0334237594596389']
 ['MINS_WATCHED' '0.0002922560423795785']
 ['GENDER_Male' '-0.012651125426665156']
 ['BILLING_PLATFORM_APPLE' '0.009337596580646314']
 ['BILLING_PLATFORM_DIRECT' '0.0006679229170066705']
 ['BILLING_PLATFORM_GOOGLE' '-0.009924763376422658']
 ['HAS_FREE_TRIAL_Yes' '0.07406647544350485']
 ['SUBSCRIPTION_PERIOD_One_year' '-0.236742022913836']
 ['SUBSCRIPTION_PERIOD_Two_Year' '-0.2391190631075816']
 ['BY_CAMPAIGN_Yes' '0.014015814792499201']
 ['SUBSCRIPTION_TYPE_Combo' '-0.07608918943231423']
 ['SUBSCRIPTION_TYPE_Standalone' '-0.10185046511911723']
 ['HAS_ANY_KIDS_PROFILE_STREAMING_Yes' '-0.02756664788689489']
 ['PRODUCT_

Modelo Final


In [18]:
# Following columns have categorical data types and they need to be converted to dummy variables
data_reduced = data[['NUMB_OF_PROFILES','DAYS_SINCE_LAST_STREAM','MINS_WATCHED','DISTINCT_TITLES',
                     'AGE_IN_MONTHS','CHILDREN',
                     'CHURN','HAS_FREE_TRIAL','SUBSCRIPTION_PERIOD', 'BY_CAMPAIGN','SUBSCRIPTION_TYPE','HAS_ANY_KIDS_PROFILE_STREAMING',
                     'STREAM_MOVIES','PAYMENT_METHOD', 'IS_RECONNECT_PAID_TO_PAID' , 'IS_RECONNECT_PAID_TO_TRIAL', 'IS_RECONNECT_TRIAL_TO_PAID', 
                     'IS_RECONNECT_TRIAL_TO_TRIAL', 'PURCHASE_DEVICE_TYPE'
                   ]]

dummy_variable_columns_red = ['CHURN','HAS_FREE_TRIAL','SUBSCRIPTION_PERIOD', 'BY_CAMPAIGN','SUBSCRIPTION_TYPE','HAS_ANY_KIDS_PROFILE_STREAMING',
                              'STREAM_MOVIES','PAYMENT_METHOD', 'IS_RECONNECT_PAID_TO_PAID' , 'IS_RECONNECT_PAID_TO_TRIAL', 'IS_RECONNECT_TRIAL_TO_PAID', 
                              'IS_RECONNECT_TRIAL_TO_TRIAL', 'PURCHASE_DEVICE_TYPE'
                              ]


# Following columns have continuous numerical data, which needs to be scaled before predictive model can be created.
# Hence, these columns will be transformed to normalized form.

to_transform_red = ['NUMB_OF_PROFILES','DAYS_SINCE_LAST_STREAM','MINS_WATCHED','DISTINCT_TITLES',
                    'AGE_IN_MONTHS','CHILDREN'
               ]

dummy_data_file_index = 0

In [19]:
y_red, dummy_data_red = get_dummy_data_with_output(dummy_variable_columns_red, data_reduced)

In [20]:
# Split the training and test set 7:3 
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(dummy_data_red,y_red,test_size=.3, random_state=0)

In [21]:
model_sm_red, arr_sm_red = get_sm_model(y_train_red,X_train_red, X_test_red);

                            OLS Regression Results                            
Dep. Variable:              CHURN_Yes   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     295.2
Date:                Mon, 12 Sep 2022   Prob (F-statistic):               0.00
Time:                        14:31:07   Log-Likelihood:                -1823.1
No. Observations:                7000   AIC:                             3694.
Df Residuals:                    6976   BIC:                             3859.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [15]:
# Run sklearn model on reduced data set
model, arr, intercept = get_sk_model(y_train_red,X_train_red, X_test_red);


Printing model coefficients and intercept summary for sklearn model:
 [['DISTINCT_TITLES' '-0.02363082622963319']
 ['NUMB_OF_PROFILES' '0.0039243880213148405']
 ['DAYS_SINCE_LAST_STREAM' '0.03206416668947616']
 ['MINS_WATCHED' '0.00018978859345581802']
 ['AGE_IN_MONTHS' '0.005538837322988967']
 ['CHILDREN' '-0.006247584368773703']
 ['BILLING_PLATFORM_APPLE' '0.010518049832089822']
 ['BILLING_PLATFORM_DIRECT' '0.0015558281975204774']
 ['BILLING_PLATFORM_GOOGLE' '-0.008717455853013442']
 ['GENDER_Male' '-0.006175342009045179']
 ['HAS_FREE_TRIAL_Yes' '0.07281730349326906']
 ['SUBSCRIPTION_PERIOD_One_year' '-0.2362473233815991']
 ['SUBSCRIPTION_PERIOD_Two_Year' '-0.2391951352052253']
 ['BY_CAMPAIGN_Yes' '0.01391439221785154']
 ['SUBSCRIPTION_TYPE_Combo' '-0.03290220621216633']
 ['SUBSCRIPTION_TYPE_Standalone' '-0.09842670420525843']
 ['HAS_ANY_KIDS_PROFILE_STREAMING_Yes' '-0.027365502845767758']
 ['STREAM_SERIES_Yes' '0.041990327770665116']
 ['STREAM_MOVIES_Yes' '0.010007361587866897']
 [

In [16]:
equation = 'y = '
for ar in arr:
    eq = str(round(float(ar[1]),3))+' x '+str(ar[0])
    if eq.startswith('-'):
        equation = equation + ' ' + eq
    else:
        equation = equation + ' + ' +eq
    
print (equation + ' ' + str(intercept))

y =  -0.024 x DISTINCT_TITLES + 0.004 x NUMB_OF_PROFILES + 0.032 x DAYS_SINCE_LAST_STREAM + 0.0 x MINS_WATCHED + 0.006 x AGE_IN_MONTHS -0.006 x CHILDREN + 0.011 x BILLING_PLATFORM_APPLE + 0.002 x BILLING_PLATFORM_DIRECT -0.009 x BILLING_PLATFORM_GOOGLE -0.006 x GENDER_Male + 0.073 x HAS_FREE_TRIAL_Yes -0.236 x SUBSCRIPTION_PERIOD_One_year -0.239 x SUBSCRIPTION_PERIOD_Two_Year + 0.014 x BY_CAMPAIGN_Yes -0.033 x SUBSCRIPTION_TYPE_Combo -0.098 x SUBSCRIPTION_TYPE_Standalone -0.027 x HAS_ANY_KIDS_PROFILE_STREAMING_Yes + 0.042 x STREAM_SERIES_Yes + 0.01 x STREAM_MOVIES_Yes + 0.02 x PAYMENT_METHOD_Credit_Card_automatic + 0.039 x PAYMENT_METHOD_Electronic_Check + 0.021 x PAYMENT_METHOD_Mailed_Check -0.012036226330024657
